In [ ]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

def load_split(split, image_size=(128, 128)):
    """
    Charge les images depuis un répertoire spécifié (train, val, test).
    Args:
        split (str): 'train', 'val', ou 'test'
        image_size (tuple): Taille cible des images (hauteur, largeur)
    Returns:
        np.array: Tableau des images aplaties
        np.array: Tableau des étiquettes
    """
    folder_normal = f'chest_Xray/{split}/NORMAL'
    folder_pneumonia = f'chest_Xray/{split}/PNEUMONIA'
    images = []
    labels = []

    # Chargement des images NORMAL
    for filename in os.listdir(folder_normal):
        path = os.path.join(folder_normal, filename)
        try:
            with Image.open(path) as img:
                img = img.convert('L')  # Conversion en niveaux de gris
                img = img.resize(image_size)  # Redimensionnement
                img_array = np.array(img).flatten() / 255.0  # Aplatissement et normalisation
                images.append(img_array)
                labels.append(0)  # Étiquette pour NORMAL
        except Exception as e:
            print(f"Erreur lors du chargement de l'image {path} : {e}")

    # Chargement des images PNEUMONIA
    for filename in os.listdir(folder_pneumonia):
        path = os.path.join(folder_pneumonia, filename)
        try:
            with Image.open(path) as img:
                img = img.convert('L')
                img = img.resize(image_size)
                img_array = np.array(img).flatten() / 255.0
                images.append(img_array)
                if 'bacteria' in filename.lower():
                    labels.append(1)  # Pneumonie bactérienne
                elif 'virus' in filename.lower():
                    labels.append(2)  # Pneumonie virale
                else:
                    print(f"Avertissement : Type de pneumonie inconnu pour '{filename}'")
                    labels.append(-1)  # Étiquette temporaire pour les cas inconnus
        except Exception as e:
            print(f"Erreur lors du chargement de l'image {path} : {e}")

    return np.array(images), np.array(labels)

# Chargement des données
X_train, y_train = load_split('train')
X_val, y_val = load_split('val')
X_test, y_test = load_split('test')

# Encodage One-Hot des labels pour la régression
encoder = OneHotEncoder(sparse=False)
y_train_enc = encoder.fit_transform(y_train.reshape(-1, 1))
y_val_enc = encoder.transform(y_val.reshape(-1, 1))

# Initialisation et entraînement du modèle
pls = PLSRegression(n_components=10)
pls.fit(X_train, y_train_enc)

# Prédiction sur le jeu de test
Y_pred = pls.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

# Évaluation
print("Rapport de classification :")
print(classification_report(y_test, y_pred))

print("Matrice de confusion :")
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=encoder.categories_[0], yticklabels=encoder.categories_[0])
plt.xlabel("Prédits")
plt.ylabel("Vérité")
plt.title("Matrice de confusion - PLSRegression")
plt.show()

# Sauvegarde du modèle
joblib.dump(pls, "pls_model.joblib")